<a href="https://colab.research.google.com/github/alex8575/trainning-html5/blob/master/Aplicaci%C3%B3n_de_Diagn%C3%B3stico_Financiero.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
"""
Aplicación de escritorio (Tkinter + Matplotlib + ReportLab) para el Diagnóstico FinHealth Score.

Funcionalidades:
1. Aplica el cuestionario de 8 preguntas (4 secciones, 2 preguntas c/u).
2. Calcula el score por sección y el score final.
3. Muestra el diagnóstico y una gráfica de barras de los scores por sección.
4. Permite generar un PDF detallado de las respuestas y resultados.
5. Permite enviar los resultados por correo electrónico (Requiere credenciales).

Instalaciones necesarias (además de Python):
pip install matplotlib
pip install reportlab
"""

import tkinter as tk
from tkinter import ttk, messagebox, filedialog
import csv
from datetime import datetime
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# --- Librerías Adicionales para Gráficos y PDF ---
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from reportlab.pdfgen import canvas as pdf_canvas
from reportlab.lib.pagesizes import letter
from reportlab.lib.units import inch
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import Paragraph, SimpleDocTemplate, Spacer, Table, TableStyle
from reportlab.lib import colors

# --- Configuración del Cuestionario y Puntuación ---
# Formato: (Texto de la pregunta, [(Opción 1, valor), (Opción 2, valor), ...])
QUESTIONS = [
    # Sección 1: GASTOS (Spend)
    ("1. ¿Cuál fue el comportamiento de tus gastos totales con respecto al ingreso familiar en los últimos 12 meses?",
     [("Mucho menores que los ingresos (100 pts)", 100), ("Un poco menores que los ingresos (75 pts)", 75), ("Aproximadamente iguales (50 pts)", 50), ("Un poco mayores que los ingresos (25 pts)", 25), ("Mucho mayores que los ingresos (0 pts)", 0)]),

    ("2. ¿Con qué puntualidad se han pagado las cuentas en tu hogar en los últimos 12 meses?",
     [("Todas a tiempo (100 pts)", 100), ("Casi todas a tiempo (60 pts)", 60), ("La mayoría a tiempo (40 pts)", 40), ("Algunas a tiempo (20 pts)", 20), ("Muy pocas a tiempo (0 pts)", 0)]),

    # Sección 2: AHORROS (Save)
    ("3. ¿Durante cuánto tiempo podrías vivir con tu dinero líquido actual, sin recurrir a créditos?",
     [("6 meses o más (100 pts)", 100), ("3 a 5 meses (75 pts)", 75), ("1 a 2 meses (50 pts)", 50), ("1 a 3 semanas (25 pts)", 25), ("Menos de 1 semana (0 pts)", 0)]),

    ("4. ¿Qué tan confiado te sientes de que estás tomando las acciones necesarias para alcanzar tus metas de largo plazo?",
     [("Muy confiado (100 pts)", 100), ("Moderadamente confiado (75 pts)", 75), ("Algo confiado (50 pts)", 50), ("Poco confiado (25 pts)", 25), ("Nada confiado (0 pts)", 0)]),

    # Sección 3: ENDEUDARSE (Borrow)
    ("5. Pensando en todas las deudas del hogar, ¿cuál describe mejor tu situación?",
     [("No tenemos deuda (100 pts)", 100), ("Deuda manejable (85 pts)", 85), ("Un poco más de lo que se puede pagar (40 pts)", 40), ("Es más de lo que se puede pagar (0 pts)", 0)]),

    ("6. ¿Cómo calificarías tu score en buró de crédito o puntaje crediticio?",
     [("Excelente (100 pts)", 100), ("Muy bueno (80 pts)", 80), ("Bueno (60 pts)", 60), ("Regular (40 pts)", 40), ("Malo / No lo sé (0 pts)", 0)]),

    # Sección 4: PLANIFICAR (Plan)
    ("7. ¿Qué tan confiado estás en que los seguros contratados y vigentes te brindarían suficiente apoyo en caso de emergencia?",
     [("Muy confiado (100 pts)", 100), ("Moderadamente confiado (75 pts)", 75), ("Algo confiado (50 pts)", 50), ("Poco confiado (25 pts)", 25), ("Nada confiado (0 pts)", 0)]),

    ("8. ¿Hasta qué punto estás de acuerdo con la afirmación: 'Mi hogar planea con antelación sus finanzas'?",
     [("Totalmente de acuerdo (100 pts)", 100), ("Algo de acuerdo (65 pts)", 65), ("Ni de acuerdo ni en desacuerdo (35 pts)", 35), ("Algo en desacuerdo (15 pts)", 15), ("Totalmente en desacuerdo (0 pts)", 0)])
]

SECTION_INDEX = {
    'Gastos': (0, 1),
    'Ahorros': (2, 3),
    'Endeudarse': (4, 5),
    'Planificar': (6, 7)
}

# --- Clase Principal de la Aplicación ---
class FinHealthApp(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Diagnóstico de Salud Financiera - FinHealth Score")
        self.geometry("1200x850")
        self.resizable(True, True)

        # Variables para almacenar la última respuesta y scores
        self._last_result = None
        self._last_section_scores = None

        self.create_widgets()

    def create_widgets(self):
        # Frame principal para contener todo
        main_frame = ttk.Frame(self, padding="10 10 10 10")
        main_frame.pack(fill=tk.BOTH, expand=True)

        # Panel de Preguntas (Izquierda)
        left_panel = ttk.Frame(main_frame, padding="5")
        left_panel.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=5, pady=5)

        # Panel de Resultados y Gráfica (Derecha)
        right_panel = ttk.Frame(main_frame, padding="5")
        right_panel.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True, padx=5, pady=5)

        # --- Panel Izquierdo: Datos y Preguntas ---

        # Datos personales
        personal_frame = ttk.LabelFrame(left_panel, text="Datos del Evaluado y Envío", padding=10)
        personal_frame.pack(fill=tk.X, padx=4, pady=6)

        data_fields = [
            ("Nombre:", "name_entry"),
            ("Email Evaluado:", "email_entry"),
            ("Email Destino:", "target_email"),
            ("Contraseña Remitente (Gmail App Pass):", "email_pass", True) # Show='*'
        ]

        for i, (label_text, attr_name, is_password) in enumerate(data_fields):
            is_password = is_password if len(data_fields[i]) > 2 else False
            ttk.Label(personal_frame, text=label_text).grid(row=i, column=0, sticky=tk.W, padx=5, pady=2)
            entry = ttk.Entry(personal_frame, width=35, show='*' if is_password else '')
            setattr(self, attr_name, entry)
            entry.grid(row=i, column=1, padx=5, pady=2, sticky=tk.EW)

        personal_frame.grid_columnconfigure(1, weight=1)


        # Scrollable area for questions
        q_frame_container = ttk.LabelFrame(left_panel, text="Cuestionario FinHealth Score", padding=5)
        q_frame_container.pack(fill=tk.BOTH, expand=True, padx=4, pady=6)

        canvas = tk.Canvas(q_frame_container)
        canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        scrollbar = ttk.Scrollbar(q_frame_container, orient="vertical", command=canvas.yview)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        canvas.configure(yscrollcommand=scrollbar.set)

        questions_frame = ttk.Frame(canvas)
        canvas.create_window((0, 0), window=questions_frame, anchor='nw')

        self.vars = []
        self.response_texts = [] # Almacena el texto de la opción seleccionada

        section_titles = ['GASTOS (Spend)', 'AHORROS (Save)', 'ENDEUDARSE (Borrow)', 'PLANIFICAR (Plan)']

        for i, (qtext, options) in enumerate(QUESTIONS):
            section_index = i // 2

            if i % 2 == 0:
                # Encabezado de la sección
                sec_label = ttk.Label(questions_frame, text=f"--- Sección {section_index+1}: {section_titles[section_index]} ---",
                                      font=('Arial', 10, 'bold'), foreground='darkblue')
                sec_label.pack(fill=tk.X, padx=6, pady=(10, 5))

            qframe = ttk.LabelFrame(questions_frame, text=f"Pregunta {i+1}", padding=8)
            qframe.pack(fill=tk.X, padx=6, pady=4)
            ttk.Label(qframe, text=qtext, wraplength=550, justify=tk.LEFT).pack(anchor=tk.W)

            var = tk.IntVar(value=-1)
            self.vars.append(var)

            # Almacena el texto de la pregunta y sus opciones
            self.response_texts.append(None)

            opts_frame = ttk.Frame(qframe)
            opts_frame.pack(anchor=tk.W, pady=4)
            for j, (opt_text, opt_val) in enumerate(options):
                # Crea una función lambda para manejar el cambio de selección y guardar el texto
                def on_select(v=var, text=opt_text, index=i):
                    v.set(opt_val) # Asegura que el valor se establezca
                    self.response_texts[index] = text

                rb = ttk.Radiobutton(opts_frame, text=opt_text, variable=var, value=opt_val, command=on_select)
                rb.pack(anchor=tk.W, padx=4, pady=2)

        questions_frame.update_idletasks()
        canvas.config(scrollregion=canvas.bbox("all"))
        canvas.bind('<Configure>', lambda e: canvas.configure(scrollregion = canvas.bbox("all")))


        # Botones de acción
        btn_frame = ttk.Frame(left_panel, padding=6)
        btn_frame.pack(fill=tk.X, pady=8)

        calc_btn = ttk.Button(btn_frame, text="Calcular Score", command=self.calculate)
        calc_btn.pack(side=tk.LEFT, padx=5, pady=5)

        pdf_btn = ttk.Button(btn_frame, text="Generar PDF", command=self.generate_pdf)
        pdf_btn.pack(side=tk.LEFT, padx=5, pady=5)

        email_btn = ttk.Button(btn_frame, text="Enviar por Correo", command=self.send_email)
        email_btn.pack(side=tk.LEFT, padx=5, pady=5)

        reset_btn = ttk.Button(btn_frame, text="Reiniciar", command=self.reset_all)
        reset_btn.pack(side=tk.RIGHT, padx=5, pady=5)

        # --- Panel Derecho: Resultados y Gráfica ---

        # Área de resultados textuales
        result_text_frame = ttk.LabelFrame(right_panel, text="Diagnóstico y Score Final", padding=10)
        result_text_frame.pack(fill=tk.X, padx=4, pady=6)
        self.result_text = tk.Text(result_text_frame, height=10, wrap=tk.WORD, font=('Courier', 10))
        self.result_text.pack(fill=tk.BOTH, expand=True)
        self.result_text.configure(state=tk.DISABLED)

        # Área de la gráfica (inicialmente vacía)
        graph_frame = ttk.LabelFrame(right_panel, text="Score por Sección", padding=10)
        graph_frame.pack(fill=tk.BOTH, expand=True, padx=4, pady=6)
        self.graph_frame = graph_frame
        self.fig = None
        self.canvas_widget = None

    # --- Lógica de Cálculo y Resultados ---
    def calculate(self):
        # 1. Validación
        unanswered = [i + 1 for i, var in enumerate(self.vars) if var.get() == -1]
        if unanswered:
            messagebox.showwarning("Faltan respuestas", f"Por favor responde las preguntas: {', '.join(map(str, unanswered))}.")
            return

        # 2. Cálculo de Scores por Sección
        section_scores = {}
        for section, (i1, i2) in SECTION_INDEX.items():
            v1 = self.vars[i1].get()
            v2 = self.vars[i2].get()
            avg = (v1 + v2) / 2.0
            section_scores[section] = avg
        self._last_section_scores = section_scores

        # 3. Cálculo del Score Final
        final_score = sum(section_scores.values()) / len(section_scores)

        # 4. Clasificación
        if 0 <= final_score <= 39:
            category = "Financieramente vulnerable"
            advice = "Es urgente revisar tus hábitos de gasto y endeudamiento. Enfócate en crear un fondo de emergencia."
        elif 40 <= final_score <= 79:
            category = "Sobreviviendo pero no progresando"
            advice = "Tienes una base sólida, pero puedes mejorar en planificación a largo plazo y optimización de deudas."
        elif 80 <= final_score <= 100:
            category = "Saludable Financieramente"
            advice = "¡Felicidades! Mantén y optimiza tu planificación, ahorros y estrategias de inversión."
        else:
            category = "Error de cálculo"
            advice = ""

        # 5. Formato de Resultados Textuales
        now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        name = self.name_entry.get().strip()
        email = self.email_entry.get().strip()

        lines = [
            "--- DIAGNÓSTICO DE SALUD FINANCIERA ---",
            f"Evaluado: {name} ({email})",
            f"Fecha de Evaluación: {now}",
            "",
            "SCORES POR SECCIÓN (Promedio de 0 a 100):",
        ]
        for sec, sc in section_scores.items():
            lines.append(f"  - {sec}: {sc:.1f} pts")
        lines.extend([
            "",
            "--- RESULTADO FINAL ---",
            f"SCORE FINHEALTH: {final_score:.1f} pts",
            f"CLASIFICACIÓN: {category}",
            "",
            f"Recomendación: {advice}",
            "-------------------------------------"
        ])

        self.result_text.configure(state=tk.NORMAL)
        self.result_text.delete('1.0', tk.END)
        self.result_text.insert(tk.END, '\n'.join(lines))
        self.result_text.configure(state=tk.DISABLED)

        # Almacena el resultado para PDF/Correo
        self._last_result = {
            'timestamp': now,
            'name': name,
            'email': email,
            'section_scores': section_scores,
            'final_score': final_score,
            'category': category,
            'advice': advice,
            'responses': [self.response_texts[i] for i in range(len(QUESTIONS))]
        }

        # 6. Generar Gráfica de Barras
        self.plot_results(section_scores)
        messagebox.showinfo("Cálculo Finalizado", f"Score Final: {final_score:.1f} pts\nClasificación: {category}")

    # --- Funciones de Gráfica ---
    def plot_results(self, scores):
        # Limpiar gráfica anterior si existe
        if self.fig:
            plt.close(self.fig)
            self.canvas_widget.destroy()

        sections = list(scores.keys())
        points = list(scores.values())

        self.fig = plt.Figure(figsize=(5, 4), dpi=100)
        ax = self.fig.add_subplot(111)

        # Colores basados en el score
        colors = ['red' if p < 40 else 'orange' if p < 80 else 'green' for p in points]

        ax.bar(sections, points, color=colors)
        ax.set_ylim(0, 100)
        ax.set_ylabel('Puntuación (pts)')
        ax.set_title('Score por Sección de Salud Financiera')
        ax.tick_params(axis='x', rotation=15)

        # Agregar etiquetas de valor a las barras
        for i, v in enumerate(points):
            ax.text(i, v + 2, f"{v:.1f}", ha='center', va='bottom', fontsize=9, color='black')

        # Integrar Matplotlib en Tkinter
        self.canvas = FigureCanvasTkAgg(self.fig, master=self.graph_frame)
        self.canvas_widget = self.canvas.get_tk_widget()
        self.canvas_widget.pack(fill=tk.BOTH, expand=True)
        self.canvas.draw()

    # --- Funciones de Exportación y Correo ---
    def generate_pdf(self):
        if not self._last_result:
            messagebox.showwarning("Sin resultado", "Calcula el resultado antes de generar el PDF.")
            return

        fname = filedialog.asksaveasfilename(defaultextension='.pdf', filetypes=[('PDF files', '*.pdf')], title='Guardar resultados como PDF')
        if not fname:
            return

        # 1. Preparar datos y estilos
        doc = SimpleDocTemplate(fname, pagesize=letter)
        styles = getSampleStyleSheet()
        story = []

        # Estilos personalizados
        style_title = styles['Title']
        style_title.alignment = 1 # Center
        style_heading = styles['h2']
        style_heading.spaceAfter = 10
        style_normal = styles['Normal']
        style_question = ParagraphStyle('Question', parent=style_normal, fontName='Helvetica-Bold', fontSize=10, leading=14)
        style_response = ParagraphStyle('Response', parent=style_normal, fontName='Helvetica', fontSize=10, leading=14, leftIndent=20)

        # 2. Encabezado
        story.append(Paragraph("<u>Evaluación de Finanzas Personales - FinHealth Score®</u>", style_title))
        story.append(Spacer(1, 0.2 * inch))
        story.append(Paragraph(f"<b>Nombre:</b> {self._last_result['name']}", style_normal))
        story.append(Paragraph(f"<b>Email:</b> {self._last_result['email']}", style_normal))
        story.append(Paragraph(f"<b>Fecha:</b> {self._last_result['timestamp']}", style_normal))
        story.append(Spacer(1, 0.4 * inch))

        # 3. Respuestas Detalladas
        story.append(Paragraph("--- HOJA DE RESPUESTAS DETALLADA ---", style_heading))

        for i, (qtext, options) in enumerate(QUESTIONS):
            story.append(Paragraph(f"<b>Pregunta {i+1}:</b> {qtext}", style_question))
            # Obtener el texto de la respuesta guardada
            response_text = self._last_result['responses'][i]
            story.append(Paragraph(f"Respuesta seleccionada: <i>{response_text}</i>", style_response))
            story.append(Spacer(1, 0.1 * inch))

        story.append(Spacer(1, 0.5 * inch))

        # 4. Tabla de Scores por Sección
        story.append(Paragraph("--- RESUMEN DE SCORES Y RESULTADO FINAL ---", style_heading))

        # Datos para la tabla
        table_data = [['SECCIÓN', 'PUNTUACIÓN OBTENIDA (0-100)']]
        for sec, score in self._last_result['section_scores'].items():
            table_data.append([sec, f"{score:.1f} pts"])

        # Fila de Score Final
        table_data.append(['<b>SCORE FINAL</b>', f'<b>{self._last_result["final_score"]:.1f} pts</b>'])

        table_style = TableStyle([
            ('BACKGROUND', (0, 0), (-1, 0), colors.darkblue),
            ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
            ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
            ('ALIGN', (0, 0), (0, -1), 'LEFT'), # Alinear nombres de sección a la izquierda
            ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
            ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
            ('BACKGROUND', (0, 1), (-1, -2), colors.beige),
            ('GRID', (0, 0), (-1, -1), 1, colors.black),
            ('FONTNAME', (0, -1), (-1, -1), 'Helvetica-Bold'),
            ('BACKGROUND', (0, -1), (-1, -1), colors.lightgrey),
        ])

        table = Table(table_data, colWidths=[2.5 * inch, 3 * inch])
        table.setStyle(table_style)
        story.append(table)
        story.append(Spacer(1, 0.4 * inch))

        # 5. Clasificación y Recomendación
        story.append(Paragraph("--- DIAGNÓSTICO FINAL ---", style_heading))
        story.append(Paragraph(f"<b>CLASIFICACIÓN OBTENIDA:</b> {self._last_result['category']}", style_question))
        story.append(Paragraph(f"<b>Recomendación:</b> {self._last_result['advice']}", style_normal))

        # Construir PDF
        try:
            doc.build(story)
            messagebox.showinfo("PDF Generado", f"PDF de resultados guardado en: {fname}")
        except Exception as e:
            messagebox.showerror("Error de PDF", f"No se pudo generar el PDF. Asegúrate de tener ReportLab instalado. Error: {e}")

    def send_email(self):
        if not self._last_result:
            messagebox.showwarning("Sin resultado", "Calcula el resultado antes de enviar.")
            return

        # 1. Obtener credenciales
        sender = self.email_entry.get().strip()
        password = self.email_pass.get().strip()
        recipient = self.target_email.get().strip()

        if not sender or not password or not recipient:
            messagebox.showwarning("Campos incompletos", "Completa los campos de Email Remitente, Contraseña y Email Destino.")
            return

        # Nota importante: Para Gmail, se necesita una 'Contraseña de Aplicación' (App Password), no la contraseña normal de la cuenta.

        # 2. Construir el mensaje
        msg = MIMEMultipart()
        msg['From'] = sender
        msg['To'] = recipient
        msg['Subject'] = 'Resultados de tu Diagnóstico de Salud Financiera'

        # Contenido del cuerpo del correo (texto simple)
        body = self.result_text.get('1.0', tk.END)
        msg.attach(MIMEText(body, 'plain', 'utf-8'))

        # 3. Intentar el envío
        try:
            # Usar 'smtp.gmail.com' para el ejemplo.
            server = smtplib.SMTP('smtp.gmail.com', 587)
            server.starttls()
            server.login(sender, password)
            server.send_message(msg)
            server.quit()
            messagebox.showinfo("Correo enviado", f"Resultados enviados exitosamente a {recipient}.")
        except smtplib.AuthenticationError:
            messagebox.showerror("Error de Autenticación", "Fallo de inicio de sesión. Revisa que tu Email Remitente y Contraseña (App Password de Gmail) sean correctos.")
        except Exception as e:
            messagebox.showerror("Error de Conexión", f"No se pudo enviar el correo. Verifica tu conexión o la configuración del servidor SMTP.\nDetalles: {e}")

    def reset_all(self):
        # Reiniciar variables de las preguntas
        for var in self.vars:
            var.set(-1)
        self.response_texts = [None] * len(QUESTIONS)

        # Reiniciar campos de texto
        self.name_entry.delete(0, tk.END)
        self.email_entry.delete(0, tk.END)
        # No reiniciar target_email y password para comodidad si se usa varias veces
        # self.target_email.delete(0, tk.END)
        # self.email_pass.delete(0, tk.END)

        # Reiniciar área de resultados
        self.result_text.configure(state=tk.NORMAL)
        self.result_text.delete('1.0', tk.END)
        self.result_text.configure(state=tk.DISABLED)

        # Reiniciar variables de estado
        self._last_result = None
        self._last_section_scores = None

        # Limpiar gráfica
        if self.fig:
            plt.close(self.fig)
            self.canvas_widget.destroy()
            self.fig = None
            self.canvas_widget = None
            messagebox.showinfo("Reiniciado", "La aplicación ha sido reiniciada. Puedes empezar un nuevo cuestionario.")


if __name__ == '__main__':
    app = FinHealthApp()
    app.mainloop()

TclError: no display name and no $DISPLAY environment variable